In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
from torchvision import datasets, transforms
from tqdm import tqdm
import os
import numpy as np
import sys

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
sys.path.append('/content/drive/MyDrive/Colab Notebooks/DnnSecurity/Attack/BadNet')
from dataset_mnist import MyDataset
from model import WideResNet

In [17]:
def train(net, dl, criterion, opt):
    running_loss = 0
    cnt = 0
    net.train()
    for i, data in tqdm(enumerate(dl)):
        opt.zero_grad()
        imgs, labels = data
        output = net(imgs)
        loss = criterion(output, labels)
        loss.backward()
        opt.step()
        cnt = i
        running_loss += loss
    return running_loss / cnt

def eval(net, dl, batch_size=64):
    cnt = 0
    ret = 0
    net.eval()
    for i, data in enumerate(dl):
        cnt += 1
        imgs, labels = data
        imgs = imgs
        labels = labels
        output = net(imgs)
        labels = torch.argmax(labels, dim=1)
        output = torch.argmax(output, dim=1)
        ret += torch.sum(labels == output)
    return int(ret) / (cnt * batch_size)

In [29]:
print("start")
# Compile
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

depth = 16
num_classes =10
widen_factor = 4
model = WideResNet(depth, num_classes, widen_factor)

model = WideResNet(depth, num_classes).to(device)
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/DnnSecurity/Attack/model_MNIST.pth"):
    model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/DnnSecurity/Attack/model_MNIST.pth", map_location=device))
criterion = nn.MSELoss()
sgd = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
epoch = 3

# dataset
transform = transforms.Compose([transforms.Grayscale()])

test_data = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/DnnSecurity/Attack/Dataset/mnist/28_28/testing/", transform=transform)
train_data = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/DnnSecurity/Attack/Dataset/mnist/28_28/training/", transform=transform)

#train_data = datasets.MNIST(root="./data/", train=True, download=True)
#test_data = datasets.MNIST(root="./data/", train=False, download=True)

train_data = MyDataset(train_data, 0, portion=0.1, mode="train", device=device)

test_data_orig = MyDataset(test_data, 0, portion=0, mode="train", device=device)
test_data_trig = MyDataset(test_data, 0, portion=1, mode="test", device=device)

train_data_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_data_orig_loader = DataLoader(dataset=test_data_orig, batch_size=64, shuffle=True)
test_data_trig_loader = DataLoader(dataset=test_data_trig, batch_size=64, shuffle=True)

# train
print("start training: ")
for i in range(epoch):
    loss_train = train(model, train_data_loader, criterion, sgd)
    acc_train = eval(model, train_data_loader)
    acc_test_orig = eval(model, test_data_orig_loader, batch_size=64)
    acc_test_trig = eval(model, test_data_trig_loader, batch_size=64)
    print(f"epoch{i + 1}   loss: {loss_train:.5f}  training accuracy: {acc_train:.5f}  testing Orig accuracy: {acc_test_orig:.5f}  testing Trig accuracy: {acc_test_trig:.5f}")
    torch.save(model.state_dict(), "./model_MNIST.pth")
print("finish training")

start


FileNotFoundError: Found no valid file for the classes 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, MNIST. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp